In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import time
import copy
import wandb

In [ ]:
wandb.login()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [11]:
wandb.init(project="resnet50_finetune", config={
    "model": "resnet50",
    "batch_size": 64,
    "learning_rate": 0.001,
    "num_epochs": 20,
    "strategy": "unfreeze_layer4_fc",
    "optimizer": "Adam",
    "image_size": 224
})
config = wandb.config

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 input
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

In [14]:
data_dir = '/kaggle/input/inaturalist1/inaturalist_12K/train'
# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Get class labels for stratified split
targets = np.array(full_dataset.targets)

# Stratified Split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(np.zeros(len(targets)), targets))

# Create train and validation datasets using stratified split
train_dataset = Subset(full_dataset, train_idx)
val_dataset = Subset(full_dataset, val_idx)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=4)


In [16]:
model = models.resnet50(pretrained=True)

# Freeze all layers first
for param in model.parameters():
    param.requires_grad = False
# Unfreeze layer4 and fc
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True
model = model.to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)

In [18]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 20)

        model.train()
        train_loss, train_correct, total_train = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = outputs.argmax(1)
            train_loss += loss.item() * inputs.size(0)
            train_correct += (preds == labels).sum().item()
            total_train += labels.size(0)

        epoch_train_loss = train_loss / total_train
        epoch_train_acc = train_correct / total_train

        model.eval()
        val_loss, val_correct, total_val = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                preds = outputs.argmax(1)
                val_loss += loss.item() * inputs.size(0)
                val_correct += (preds == labels).sum().item()
                total_val += labels.size(0)

        epoch_val_loss = val_loss / total_val
        epoch_val_acc = val_correct / total_val

        print(f"Train Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f}")
        print(f"Val   Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f}")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": epoch_train_loss,
            "train_acc": epoch_train_acc,
            "val_loss": epoch_val_loss,
            "val_acc": epoch_val_acc
        })

In [19]:
train_model(model, train_loader, val_loader, criterion, optimizer, config.num_epochs)
wandb.finish()


Epoch 1/10
--------------------
Train Loss: 1.2520 Acc: 0.6266
Val   Loss: 0.9251 Acc: 0.6945

Epoch 2/10
--------------------
Train Loss: 0.6900 Acc: 0.7735
Val   Loss: 0.8176 Acc: 0.7400

Epoch 3/10
--------------------
Train Loss: 0.4795 Acc: 0.8457
Val   Loss: 1.0083 Acc: 0.6975

Epoch 4/10
--------------------
Train Loss: 0.3640 Acc: 0.8759
Val   Loss: 0.8925 Acc: 0.7380

Epoch 5/10
--------------------
Train Loss: 0.2625 Acc: 0.9109
Val   Loss: 1.1657 Acc: 0.7145

Epoch 6/10
--------------------
Train Loss: 0.2037 Acc: 0.9344
Val   Loss: 0.8468 Acc: 0.7600

Epoch 7/10
--------------------
Train Loss: 0.1569 Acc: 0.9492
Val   Loss: 1.0484 Acc: 0.7330

Epoch 8/10
--------------------
Train Loss: 0.1353 Acc: 0.9557
Val   Loss: 0.9612 Acc: 0.7515

Epoch 9/10
--------------------
Train Loss: 0.1115 Acc: 0.9636
Val   Loss: 1.0804 Acc: 0.7470

Epoch 10/10
--------------------
Train Loss: 0.0919 Acc: 0.9712
Val   Loss: 1.1177 Acc: 0.7535


epoch,▁▂▁▂▃▃▄▅▆▆▇█
train_acc,▁▃▄▆▇▇▇█████
train_loss,█▆▅▃▂▂▂▁▁▁▁▁
val_acc,▁▃▇█▇█▇█▇███
val_loss,█▇▂▁▂▂▃▁▃▂▃▃
epoch,10
train_acc,0.97125
train_loss,0.09195
val_acc,0.7535
val_loss,1.11774


In [20]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load test data
test_dataset = datasets.ImageFolder(root='/kaggle/input/inaturalist1/inaturalist_12K/val', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model to eval mode

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 75.25%
